In [ ]:
# First load the data
dbHermina = pd.read_pickle('../data/ephys_Germany/HerminaDatabase.')

numUnits = [1, 2, 3, 4, 5, 10, 20]
groupings = []
pcc = []
confMat = []
dataPath = '/auto/zdrive/lthomas/Code/zebra-finch-categories/data/ephys_Germany/'
for nUnit in numUnits:
    inPath = dataPath + 'HerminaDatabase%dGoodUnitPC.pkl' % nUnit
    fileIn = open(inPath,"rb")
    # groupings
    groupings.append(pk.load(fileIn))
    # conf matricies
    confMat.append(pk.load(fileIn))
    # percent correct scores for each grouping
    pcc.append(pk.load(fileIn))
    fileIn.close()

In [ ]:
# Generate pkl files for each grouping after aris are calculated, comparing against calltype

import multiprocessing,time

stims = np.unique(dbHermina.stim)  
identities_orig = [x.split('_')[0] for x in stims]
calltypes_orig = [x.split('_')[2] for x in stims]
n_pcs = 5

def cluster_analysis_short(db_sites):
    avg_responses = np.zeros((len(stims),len(db_sites)*(1+n_pcs)))
    for ind, db_site in enumerate(db_sites):
        for stim in np.unique(db_site.stim):
            stim_ind = np.where(stims==stim)
            db_site_stim = db_site[db_site.stim == stim]
            avg_responses[stim_ind,ind*(1+n_pcs)] = np.mean(db_site_stim.z)
            n_trials[stim_ind,ind] = len(db_site_stim.z)
            for i in range(n_pcs):
                avg_responses[stim_ind,ind*(1+n_pcs)+i+1] = np.mean([x[i] for x in db_site_stim.PC])
    return cluster_analysis(avg_responses,calltypes_orig,score='ari')# and_plot = True
        
#args = []


for i in range(len(groupings)):
    pccs_rand = []
    print(i)
    # accumulate all groupings of sites for this group size
    g_sites = []
    
    # go through each grouping to generate collection of sites
    for g in range(len(groupings[i])):
        # get the grouping and pcc for this grouping
        grp = groupings[i][g]
        p = pcc[i][g]
        
        pccs_rand.append(p)

        # response len will be (z,pc1,pc2...pcn) for each unit
        query = None
        sites = []
        for ind, g in enumerate(grp):
            # for each group
            s,g = g.split('_goodPlayback')
            query = (dbHermina.site == s)&(dbHermina.unit == 'goodPlayback%s'%g)
            db_site = dbHermina[query]
            sites.append(db_site.copy())
            
        # append this collection of sites to the collection of groups of sites
        g_sites.append(sites)

    print("starting multiproc for %s"%i)
    tst = time.time()
    pool = multiprocessing.Pool(12)
    results = pool.map(cluster_analysis_short,g_sites)       
    print("ending %s"%(time.time() - tst))
    #result_dict[i] = results
    pool.close()
    rd = dict()
    rd['pccs'] = pccs_rand
    rd['results'] = results
    rd['group'] = groupings[i]
    with open('../data/ephys_Germany/Groupings/Groupings_%s.pkl'%len(g_sites[0]), 'wb') as handle:
        pk.dump(rd, handle, protocol=pk.HIGHEST_PROTOCOL)


In [ ]:
# CALCULATE ari_neural_vs_aud

import glob,time
import pickle as pk
from sklearn.metrics import adjusted_rand_score,adjusted_mutual_info_score

groups = glob.glob("../data/ephys_Germany/Groupings/Groupings_*.pkl")
# make a dataframe for easy processing
dfout_add = dict({
    'neural_grouping': [],
    'ari_neural_vs_aud': []})
for gi, group_fp in enumerate(groups):
#def proc_group_pf(group_fp):
    print("Starting %s"%group_fp)
    tst = time.time()
    group = pk.load(open(group_fp,'rb'))
    pccs = group['pccs']
    results = group['results']
    neural_groupings = [r[4] for r in results]
    
    n_v_a_aris = [ adjusted_rand_score(ng[has_wavs],aud_lbl_umap) if ng is not None else 0.0 for ng in neural_groupings]
    dfout_add['neural_grouping'].extend(neural_groupings)
    dfout_add['ari_neural_vs_aud'].extend(n_v_a_aris)
    print("Finished %s in %s"%(group_fp,time.time()-tst))
    #return results[0][4], adjusted_rand_score(results[0][4][has_wavs],aud_lbl_umap)



dfout_f['neural_grouping'] =  dfout_add['neural_grouping']# = pd.DataFrame(dfout)
dfout_f['ari_neural_vs_aud'] = dfout_add['ari_neural_vs_aud']#
dfout_f.to_hdf("../data/ephys_Germany/Groupings_metadata_with_ng.hdf",'data')